In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [2]:
image_paths = sorted(glob("/tf/hayoung/datafortransformation/yolo/valid/*.png")) # 절대경로 지정 필요
txt_paths = sorted(glob("/tf/hayoung/datafortransformation/yolo/valid/*.txt")) # 절대경로 지정 필요

SEED = 42
MODEL = 'v1'
train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.1, random_state=SEED)

In [3]:
def Yolo2LabelMe(image_paths, txt_paths, type="train"):    
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape
        
        target_image_path = f"/tf/hayoung/datafortransformation/labelme/{type}/{os.path.basename(image_path)}" # 절대경로 지정 필요
        cv2.imwrite(target_image_path, source_image)
        
        if type == "test":
            continue
            
        with open(txt_path, "r") as reader:
            labelme_labels = []
            lines = reader.readlines()
            for line in lines:
                # confidence 필요 시 나중에 추가할 것
                class_id, x, y, width, height = [float(temp) for temp in line.split()]
                
                x_min = int((x - width / 2) * image_width)
                x_max = int((x + width / 2) * image_width)
                y_min = int((y - height / 2) * image_height)
                y_max = int((y + height / 2) * image_height)
                
                labelme_labels.append(f"{int(class_id)} {x_min} {y_min} {x_max} {y_min} {x_max} {y_max} {x_min} {y_max}")
        
        target_label_txt = f"/tf/hayoung/datafortransformation/labelme/{type}/{os.path.basename(txt_path)}" # 절대경로 지정 필요
        
        with open(target_label_txt, "w") as writer:
            for labelme_label in labelme_labels:
                writer.write(f"{labelme_label}\n")            

In [6]:
Yolo2LabelMe(train_images_paths, train_txt_paths, "train")
Yolo2LabelMe(valid_images_paths, valid_txt_paths, "valid")

# Yolo2LabelMe(sorted(glob("/test/*.png")), None, "test") # 테스트 이미지 폴더 절대경로 지정 필요

100%|██████████| 65/65 [00:07<00:00,  8.66it/s]


In [9]:
# source_txts = glob(f"{MODEL}/predict/labels/*.txt")
source_txts = glob(f"/tf/hayoung/datafortransformation/labelme/valid/*.txt")


# CSV 형태 Submission이 필요한 경우 
# confidence 필요 시 나중에 추가할 것
def submitLabelmeAsCSV(source_txts):
    results = []
    for source_txt in tqdm(source_txts):
        file_name = source_txt.split("/")[-1].replace(".txt", ".png") #필요시 이미지 확장자 변경할 것
        with open(source_txt, "r") as reader:        
            lines = reader.readlines()
            for line in lines: # confidence 필요 시 나중에 추가
                line = list(map(float, line.strip().split(" ")))
                class_id = int(line[0])
                point1_x = int(line[1])
                point1_y = int(line[2])
                point2_x = int(line[3])
                point2_y = int(line[4])
                point3_x = int(line[5])
                point3_y = int(line[6])
                point4_x = int(line[7])
                point4_y = int(line[8])
                
                results.append((file_name, class_id, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y))
                
    df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
    df_submission.to_csv(f"/tf/hayoung/results/labelme_{MODEL}.csv", index=False) # 경로 나중에 변경할 것
    
submitLabelmeAsCSV(source_txts)

100%|██████████| 65/65 [00:00<00:00, 33728.78it/s]
